# EuroSAT STAC generation with the EO-TDL lib

In this demo we will do the following:
 * Generate STAC metadata for the [EuroSAT](https://github.com/phelber/EuroSAT) dataset using the EO-TDL lib
 * Convert the STAC metadata into a STACDataFrame
 * Ingest the STACDataFrame with GeoDB

In [7]:
%load_ext autoreload
%autoreload 2

%load_ext dotenv
%dotenv
from eotdl.curation import STACGenerator, UnestructuredParser
from os.path import join

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [8]:
stac_generator = STACGenerator(image_format='tif', item_parser=UnestructuredParser)

ROOT = '../../../../datasets/EuroSAT-RGB/EuroSAT/'   # Path to the root folder of the dataset, change it to your path

df = stac_generator.get_stac_dataframe(ROOT)
df.head()

,image,label,ix,collection,extensions,bands
0,../../../../datasets/EuroSAT-RGB/EuroSAT/Pastu...,Pasture,0,../../../../datasets/EuroSAT-RGB/EuroSAT/source,None,None
1,../../../../datasets/EuroSAT-RGB/EuroSAT/Resid...,Residential,1,../../../../datasets/EuroSAT-RGB/EuroSAT/source,None,None
2,../../../../datasets/EuroSAT-RGB/EuroSAT/Fores...,Forest,2,../../../../datasets/EuroSAT-RGB/EuroSAT/source,None,None
3,../../../../datasets/EuroSAT-RGB/EuroSAT/Fores...,Forest,2,../../../../datasets/EuroSAT-RGB/EuroSAT/source,None,None
4,../../../../datasets/EuroSAT-RGB/EuroSAT/River...,River,4,../../../../datasets/EuroSAT-RGB/EuroSAT/source,None,None


In [15]:
output = 'data/EuroSAT-STAC-test'
stac_generator.generate_stac_metadata(id='eurosat-stac', description='EuroSAT dataset with STAC', stac_dataframe=df, output_folder=output)

Generating source collection...


100%|██████████| 100/100 [00:00<00:00, 249.01it/s]


STACError: Not all the STAC items have been generated, please check the Item parser or the STAC dataframe. If you are using the StructuredParser, check that the images are in the correct folder structure, or try with the UnestructuredParser

In [8]:
catalog = 'data/EuroSAT-STAC-sample/catalog.json'
stac_generator.generate_stac_labels(catalog)

Generating labels collection...


100it [00:00, 2609.62it/s]


## STAC to DataFrame and viceversa

In [ ]:
from eotdl import read_stac

path = 'data/EuroSAT-STAC/catalog.json'
df = read_stac(path)
# df = STACDataFrame.from_stac_file(path)   # Alternative way to read a STAC file
df.head()

,type,id,stac_version,description,links,stac_extensions,extent,license,properties,geometry,assets,bbox,collection
0,Catalog,eurosat,1.0.0,EuroSAT dataset,"[{'rel': 'RelType.SELF', 'href': '/Users/fran/...",[],NaN,NaN,NaN,POLYGON EMPTY,NaN,NaN,NaN
1,Collection,AnnualCrop,1.0.0,Collection,"[{'rel': 'RelType.ROOT', 'href': '../catalog.j...",[],"{'spatial': {'bbox': [[0, 0, 0, 0]]}, 'tempora...",proprietary,NaN,POLYGON EMPTY,NaN,NaN,NaN
2,Collection,Pasture,1.0.0,Collection,"[{'rel': 'RelType.ROOT', 'href': '../catalog.j...",[],"{'spatial': {'bbox': [[0, 0, 0, 0]]}, 'tempora...",proprietary,NaN,POLYGON EMPTY,NaN,NaN,NaN
3,Collection,River,1.0.0,Collection,"[{'rel': 'RelType.ROOT', 'href': '../catalog.j...",[],"{'spatial': {'bbox': [[0, 0, 0, 0]]}, 'tempora...",proprietary,NaN,POLYGON EMPTY,NaN,NaN,NaN
4,Collection,PermanentCrop,1.0.0,Collection,"[{'rel': 'RelType.ROOT', 'href': '../catalog.j...",[],"{'spatial': {'bbox': [[0, 0, 0, 0]]}, 'tempora...",proprietary,NaN,POLYGON EMPTY,NaN,NaN,NaN


Now, we can convert the STACDataframe into STAC metadata again if we want.

In [ ]:
df.to_stac()

## GeoDB ingestion

Also, we can ingest it with [GeoDB](https://xcube-geodb.readthedocs.io/en/latest/core.html). By doing this, we are converting our STACDataFrame into a Collection in GeoDB.

In [ ]:
from eotdl import STACDataFrame
from dotenv import load_dotenv
import os

load_dotenv()

server_url=os.environ['SERVER_URL']
server_port=os.environ['SERVER_PORT']
client_id=os.environ['CLIENT_ID']
client_secret=os.environ['CLIENT_SECRET']
auth_aud=os.environ['AUTH_DOMAIN']
collection = 'eurosat_stac_test'

df.to_geodb(server_url, server_port, client_id, client_secret, auth_aud, collection)

Processing rows from 0 to 61


Later on, we can go back and create STAC metadata from a GeoDB Collection

In [ ]:
df_ = STACDataFrame.from_geodb(server_url, server_port, client_id, client_secret, auth_aud, collection)
df_.head()

,id,created_at,modified_at,geometry,type,stac_id,stac_version,description,links,stac_extensions,extent,license,properties,assets,bbox,collection
0,1,2023-06-12T11:04:49.660889+00:00,None,POLYGON EMPTY,Catalog,eurosat,1.0.0,EuroSAT dataset,"[{'rel': 'self', 'href': '/Users/fran/Document...",[],,,,,,
1,2,2023-06-12T11:04:49.660889+00:00,None,POLYGON EMPTY,Collection,AnnualCrop,1.0.0,Collection,"[{'rel': 'root', 'href': '../catalog.json', 't...",[],"{'spatial': {'bbox': [[0, 0, 0, 0]]}, 'tempora...",proprietary,,,,
2,3,2023-06-12T11:04:49.660889+00:00,None,POLYGON EMPTY,Collection,Pasture,1.0.0,Collection,"[{'rel': 'root', 'href': '../catalog.json', 't...",[],"{'spatial': {'bbox': [[0, 0, 0, 0]]}, 'tempora...",proprietary,,,,
3,4,2023-06-12T11:04:49.660889+00:00,None,POLYGON EMPTY,Collection,River,1.0.0,Collection,"[{'rel': 'root', 'href': '../catalog.json', 't...",[],"{'spatial': {'bbox': [[0, 0, 0, 0]]}, 'tempora...",proprietary,,,,
4,5,2023-06-12T11:04:49.660889+00:00,None,POLYGON EMPTY,Collection,PermanentCrop,1.0.0,Collection,"[{'rel': 'root', 'href': '../catalog.json', 't...",[],"{'spatial': {'bbox': [[0, 0, 0, 0]]}, 'tempora...",proprietary,,,,


And create the STAC metadata from the STACDataFrame.

In [ ]:
df_.to_stac()